In [66]:
from collections import Counter
from Bio import Entrez

In [67]:
Entrez.email = "sonettd@uw.edu"

In [97]:
blast_file = 'greengenes_blast2go_go_table.txt'
output_file = 'greengenes_blasted_lineages.txt'

In [114]:
with open(blast_file) as file:
    total = -1 #start at -1 to account for header line
    not_resulted = 0
    uncultured = 0
    identified = 0
    genera = []
    for line in file:
        total += 1
        values = line.split('\t')
        if values[3] == 'Description':
            continue
        elif values[3] in ['', '---NA---']:
            not_resulted += 1
        elif 'Uncultured' in values[3]:
            uncultured += 1
        else:
            genus = values[3].split(' ')[0]
            genus = genus.lstrip('"')
            genus = genus.lstrip("'")
            if not genus in ['Achain', 'TPA_asm:', 'Fungal']:
                genera.append(genus)
                identified += 1
            else:
                uncultured += 1
    genera = Counter(genera)

In [115]:
lineages = {}
for genus in genera:
    handle = Entrez.esearch(db = 'Taxonomy', retmax = 1, term = genus)
    record = Entrez.read(handle)
    handle.close()
    tax_id = record['IdList'][0]
    handle = Entrez.efetch(db = 'Taxonomy', id = tax_id)
    record = Entrez.read(handle)
    handle.close()
    lineage = record[0]['Lineage']
    if lineage not in lineages:
        lineages[lineage] = genera[genus]
    else:
        lineages[lineage] += genera[genus]

In [116]:
with open(output_file, 'w') as file:
    file.write(str(total) + '\tTotal blasted\n')
    file.write(str(not_resulted) + '\tNo result\n')
    file.write(str(uncultured+1) + '\tUncultured\n') #to account for deleted AChain result
    for lineage in lineages:
        line = lineage.replace('; ', '\t')
        file.write(str(lineages[lineage]) + '\t' + line + '\n')